# LangChain Expression Language

### What is LCEL?

* A declarative way to build AI pipelines in LangChain.
* Instead of long Python code, you connect components in a chain-like style.

---

### Why LCEL?

* Shorter and cleaner code.
* Easier to read and maintain.
* Works the same for synchronous, asynchronous, batch, and streaming execution.
* Good for debugging and reusing components.

---

### Basic Example

Without LCEL:

```python
prompt = PromptTemplate.from_template("Tell me a joke about {topic}")
chain = LLMChain(llm=ChatOpenAI(), prompt=prompt)
result = chain.run("cats")
```

With LCEL:

```python
chain = prompt | ChatOpenAI() | StrOutputParser()
result = chain.invoke({"topic": "cats"})
```
**prompt, ChatOpenAI(), StrOutputParser() are called runnables**

---

## LCEL Operators

* `|` → Pipe operator, connects steps.
* `.invoke()` → Run once with input.
* `.batch()` → Run for multiple inputs.
* `.stream()` → Get outputs step by step (streaming responses).

---

## Key Features

1. Composable – you can plug and play different parts.
2. Parallel and sequential execution supported.
3. Unified API – works the same for sync, async, batch, and streaming.
4. Production-ready – optimized for speed and scalability.

---

## Simple Summary

* LCEL is a short and readable way to connect LangChain components.
* It works like piping data through different steps.
* Use `|` to chain steps.
* The same code works for real-time, async, or batch calls.

---


# ➡️Piping a prompt, model, and an output parser

In [19]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser, StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import config

In [7]:
list_instructions = CommaSeparatedListOutputParser().get_format_instructions()

list_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [8]:
# Chat Template

chat_template = ChatPromptTemplate.from_messages([
    ('human', "I've recently adopted a {pet}. Could you suggest three {pet} names? \n" + list_instructions)
])

print(chat_template.messages[0].prompt.template)

I've recently adopted a {pet}. Could you suggest three {pet} names? 
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [3]:
# Chat Model

def AskGeminiAI(question):
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        google_api_key=config.gemini_api_key,
        temperature=0.7,
        max_tokens=250,
        model_kwargs={"seed": 42}
    )
    response = llm.invoke(question)
    return response.content

In [18]:
# Output parser

list_output_parser = CommaSeparatedListOutputParser()

In [28]:
# 3 step

chat_template_result = chat_template.invoke({'pet':'dog'})

chat_result = AskGeminiAI(chat_template_result)

list_output_parser.invoke(chat_result)

['Buddy', 'Luna', 'Charlie']

In [29]:
# Single line

chain = chat_template | AskGeminiAI | list_output_parser

chain.invoke({'pet':'dog'})

['Buddy', 'Luna', 'Charlie']

# ➡️Batching

In [30]:
chat_template = ChatPromptTemplate.from_messages([
    ('human', 
     "I've recently adopted a {pet} which is a {breed}. Could you suggest several training tips?")
])

In [31]:
chain = chat_template | AskGeminiAI

In [32]:
chain.invoke({'pet':'dog', 'breed':'shepherd'})

'Congratulations on adopting a Shepherd! They are intelligent and loyal dogs, but can also be energetic and require consistent training. Here are some training tips tailored to Shepherds:\n\n**1. Establish Yourself as the Leader (Positive Reinforcement Focus):**\n\n*   **Consistency is Key:** Shepherds thrive on structure and routine. Be consistent with your commands, rules, and expectations. Everyone in the household should use the same commands and enforce the same rules.\n*   **Positive Reinforcement:** Shepherds respond very well to positive reinforcement. Use treats, praise, toys, and affection to reward desired behaviors.  Avoid punishment-based methods, as they can damage your relationship and lead to anxiety or aggression.\n*   **Clear Communication:** Use clear and concise commands. Avoid long sentences or confusing jargon. Keep your tone consistent and confident.\n\n**2. Basic Obedience Training:**\n\n*   **Start Early:** Begin training as soon as possible, even if your Sheph

In [37]:
%%time
chain.batch([{'pet':'dog', 'breed':'shepherd'}, 
             {'pet':'dragon', 'breed':'night fury'}])

CPU times: total: 109 ms
Wall time: 3.03 s


["Congratulations on adopting a Shepherd! They are intelligent and eager to please, but also need consistent training and mental stimulation. Here are some training tips specifically geared towards Shepherds:\n\n**1. Early Socialization is Key:**\n\n*   **Expose your Shepherd to a variety of sights, sounds, people, and other dogs.**  Do this in a controlled and positive manner.  Think puppy classes, walks in different environments, and meeting friendly, well-behaved dogs.\n*   **Socialization window:**  The critical socialization period is between 8 and 16 weeks of age.  While it's ideal to start during this time, socialization is important throughout their life.\n*   **Positive Reinforcement:**  Use treats, praise, and toys to reward positive interactions.  Never force an interaction if your dog is scared or uncomfortable.\n\n**2. Basic Obedience is Essential:**\n\n*   **Start with the basics:**  Sit, stay, come, down, leave it, and heel.  Use positive reinforcement methods.\n*   **Co

In [38]:
%%time
chain.invoke({'pet':'dog', 'breed':'shepherd'})

CPU times: total: 62.5 ms
Wall time: 2.63 s


'Congratulations on adopting a shepherd! They\'re intelligent and eager to please, but also need consistent training and mental stimulation. Here are some training tips specific to shepherds:\n\n**1. Focus on Positive Reinforcement:**\n\n*   **Treats, Praise, and Toys:** Shepherds respond very well to positive reinforcement. Use high-value treats (small, smelly, and irresistible!), enthusiastic praise, and favorite toys as rewards for good behavior.\n*   **Avoid Punishment:** Punishment can damage your bond and lead to fear-based aggression. Focus on rewarding the behaviors you want to see.\n*   **Clicker Training:** Consider using a clicker. It\'s a great way to mark the exact moment your dog performs the desired behavior, followed by a reward.\n\n**2. Start Early and Be Consistent:**\n\n*   **Basic Obedience:** Teach basic commands like "sit," "stay," "down," "come," and "leave it." Start in a quiet environment and gradually introduce distractions.\n*   **Consistency is Key:** Everyo

In [39]:
%%time
chain.invoke({'pet':'dragon', 'breed':'night fury'})

CPU times: total: 78.1 ms
Wall time: 3.86 s


"Congratulations on adopting a Night Fury! They are incredibly intelligent, loyal, and powerful dragons, but also require a special approach to training. Here are some training tips, keeping in mind their unique personality:\n\n**Building Trust and Connection (Crucial for Night Furies):**\n\n*   **Patience and Understanding:** This is paramount. Night Furies are naturally wary and independent. Rushing the process will only push them away. Go slow, be gentle, and respect their boundaries.\n*   **Positive Reinforcement:** Absolutely essential. Forget punishment. Focus on rewarding desired behaviors with praise, affection (scratches under the chin are often appreciated!), and treats (more on that later).\n*   **Non-Verbal Communication:** Pay close attention to your Night Fury's body language. Learn to read their ear positions, tail movements, and vocalizations. They are excellent communicators, but you need to listen.\n*   **Bonding Time:** Spend quality time with your Night Fury without

### 1. **CPU time**

* The amount of time the CPU actually spent executing your code.
* Includes user + system CPU usage.
* If your code runs in **parallel** across multiple cores, CPU time can be **greater than wall time** (because it sums across cores).

---

### 2. **Wall time**

* The real-world elapsed time (like looking at a stopwatch).
* Starts when the cell begins execution, stops when it ends.
* Includes waiting, I/O delays, and any time the process was idle.

---

# ➡️Streaming

In [41]:
chat_template = ChatPromptTemplate.from_messages([
    ('human', 
     "I've recently adopted a {pet} which is a {breed}. Could you suggest several training tips?")
])

In [4]:
llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        google_api_key=config.gemini_api_key,
        temperature=0.7,
        max_tokens=250,
        model_kwargs={"seed": 42}
    )

In [51]:
chain = chat_template | llm

In [52]:
%%time
chain.invoke({'pet':'dog', 'breed':'shepherd'})

CPU times: total: 15.6 ms
Wall time: 2.6 s


AIMessage(content="Congratulations on adopting a shepherd! They're intelligent and eager to please, which makes them generally very trainable. Here are some training tips, broken down into categories, to get you started:\n\n**1. Understanding Your Shepherd's Breed:**\n\n*   **High Energy & Intelligence:** Shepherds need a job. They were bred to herd and work, so they need mental and physical stimulation to prevent boredom and destructive behaviors.\n*   **Herding Instincts:** Be aware of their herding instincts. They may try to herd children, other pets, or even cars. Training will help manage this.\n*   **Loyalty & Protectiveness:** Shepherds are loyal and protective of their families. Socialization is crucial to ensure they are friendly and confident around strangers.\n\n**2. Basic Obedience Training:**\n\n*   **Start Early:** Even if your dog is an adult, start training immediately. Reinforce good behaviors.\n*   **Positive Reinforcement:** Use positive reinforcement methods like tr

In [53]:
%%time
chain.batch([{'pet':'dog', 'breed':'shepherd'}, 
             {'pet':'dragon', 'breed':'night fury'}])

CPU times: total: 78.1 ms
Wall time: 3.65 s


[AIMessage(content="Congratulations on adopting a shepherd! They are intelligent, loyal, and eager to please, which makes them generally good candidates for training. However, their intelligence and energy also mean they need consistent and engaging training. Here are some training tips tailored for shepherds:\n\n**1. Start Early and Be Consistent:**\n\n*   **Begin training as soon as possible.** Even if your dog is older, you can start right away.\n*   **Consistency is key.** Use the same commands and hand signals every time. Everyone in the household should use the same cues.\n*   **Keep training sessions short and frequent.** Focus on 5-10 minute sessions several times a day rather than one long, drawn-out session. This helps maintain your dog's focus.\n\n**2. Positive Reinforcement is Your Best Friend:**\n\n*   **Use treats, praise, and toys as rewards.**  Find what motivates your dog the most. High-value treats (small pieces of cheese, cooked meat, etc.) can be especially effectiv

In [61]:
response = chain.stream({'pet':'dragon', 'breed':'night fury'})

# next(response) 
# Helps to retrieve the first chunk of output generated by the model.

for i in response:
    print(i.content, end = '')

Congratulations on adopting a Night Fury! They are incredibly intelligent, loyal, and powerful dragons, but also require a specific approach to training. Here are some training tips, keeping in mind that building trust and understanding is key:

**1. Building Trust and Bonding:**

*   **Patience is paramount:** Night Furies are initially wary of humans, so don't rush the process. Let your dragon come to you.
*   **Positive Reinforcement:** Forget punishment! Focus on rewarding desired behaviors with treats, praise, and affection. Fish is their favorite, especially salmon and tuna. Learn your dragon's preferences!
*   **Gentle Approach:** Avoid loud noises, sudden movements, or anything that might startle your dragon. Speak in a calm, soothing voice.
*   **Understand Body Language:** Pay close attention to your Night Fury's body language. Learn to recognize signs of fear, stress, happiness, and curiosity. Dilated pupils, flattened ears, and a tucked tail indicate fear or discomfort. Pur

### 1. `chain.stream(...)`

* Runs the LCEL chain in **streaming mode**.
* Instead of waiting for the **full response**, it returns an **iterator (generator)**.
* Each `yield` gives you a partial output (token or chunk).

---

### 2. `next(response)`

* The first call gets the very first chunk from the model’s output.
* This is often needed to “start” the stream before looping.
* Sometimes developers use it to **inspect metadata** or headers before reading the actual content.

---

### 3. `for i in response:`

* Iterates through the rest of the streamed chunks.
* Each `i` is usually a `ChatGenerationChunk` or a token-like object.

---

### 4. `print(i.content, end='')`

* Prints the text as it streams in, without adding new lines.
* This makes the output look like the model is typing word by word.

---

### Example Flow

If the model response is:

```
"Night Furies are rare dragons known for their speed."
```

Streaming loop prints:

```
Night Furies are rare dragons known for their speed.
```

…but in reality, it arrives piece by piece:

* `Night`
* ` Furies`
* ` are`
* ` rare`
* …and so on.

---


# ➡️The Runnable and RunnableSequence classes

In [9]:
chain = chat_template | llm

In [11]:
type(chain)

langchain_core.runnables.base.RunnableSequence

In [12]:
type(chat_template)

langchain_core.prompts.chat.ChatPromptTemplate

### Runnable in LangChain

* A **Runnable** is just something that can be “run” with input and gives output.
* Examples:

  * A prompt template (`PromptTemplate`)
  * An LLM (`ChatOpenAI`, `Gemini`)
  * An output parser

All of them share the same interface: `.invoke(input)`

---

### RunnableSequence

* A **`RunnableSequence`** is a chain of Runnables connected together.
* Think of it as a **pipeline**:

  * Output of one step → becomes input to the next step.
* It is created when you use the **pipe operator (`|`)**.

---

# ➡️Piping chains and the RunnablePassthrough class

In [15]:
RunnablePassthrough().invoke([1, 2, 3])

[1, 2, 3]

### What is `RunnablePassthrough`?

* It’s a special kind of **Runnable** in LangChain.
* **Whatever input you give it, it returns the same thing as output.**
* No changes, no processing — just passes the data through.

---

### Why is it useful?

* Sometimes you want to **keep the original input** available in a chain.
* Example: You build a chain where one branch transforms the input, but another branch just needs the raw input.
* `RunnablePassthrough` helps in those cases.

---


In [35]:
chat_template_tools = ChatPromptTemplate.from_template('''
What are the five most important tools a {job title} needs?
Answer only by listing the tools.
''')

chat_template_strategy = ChatPromptTemplate.from_template('''
Considering the tools provided, develop a strategy for effectively learning and mastering them:
{tools}
''')

In [20]:
string_parser = StrOutputParser()

In [21]:
llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        google_api_key=config.gemini_api_key,
        temperature=0.7,
        max_tokens=250,
        model_kwargs={"seed": 42}
    )

In [40]:
# A chain that lists the most essential tools for a given prodession
chain_tools = chat_template_tools | llm | string_parser

# A chain that suggests effective strategies for mastering these tools
chain_strategy = chat_template_strategy | llm | string_parser

In [41]:
print(chain_tools.invoke({'job title':'data scientist'}))

1.  Python
2.  SQL
3.  R
4.  Cloud Computing (e.g., AWS, Azure, GCP)
5.  Machine Learning Frameworks (e.g., TensorFlow, PyTorch, scikit-learn)


In [42]:
print(chain_strategy.invoke({'tools':'''
1.  Python
2.  SQL
3.  R
4.  Cloud Computing (e.g., AWS, Azure, GCP)
5.  Machine Learning Frameworks (e.g., TensorFlow, PyTorch, scikit-learn)
'''}))

Okay, here's a comprehensive strategy for effectively learning and mastering the listed tools, focusing on a practical, project-based approach with continuous reinforcement and adaptation:

**Overall Learning Philosophy:**

*   **Project-Based Learning:** The most effective way to learn these tools is by applying them to real-world projects.  Start small, iterate, and gradually increase complexity.
*   **Active Learning:**  Don't just passively read documentation or watch videos. Code along, experiment, and try to break things.  Actively engage with the material.
*   **Consistent Practice:**  Even short, regular practice sessions (e.g., 30-60 minutes daily) are more effective than long, infrequent sessions.
*   **Community Engagement:**  Join online forums, communities, and attend meetups.  Ask questions, share your knowledge, and learn from others.
*   **Documentation is Your Friend:**  Learn to navigate and utilize the official documentation for each tool. It's the most reliable sour

In [45]:
# Combining two chains

chain_tools = chat_template_tools | llm | string_parser | {'tools':RunnablePassthrough()}
# The output of {'tools':RunnablePassthrough()} is dictionary in the format:
# {'tools':'''
# 1.  Python
# 2.  SQL
# 3.  R
# 4.  Cloud Computing (e.g., AWS, Azure, GCP)
# 5.  Machine Learning Frameworks (e.g., TensorFlow, PyTorch, scikit-learn)
# '''}

chain_strategy = chat_template_strategy | llm | string_parser

chain_combined = chain_tools | chain_strategy

In [44]:
print(chain_combined.invoke({'job title':'data scientist'}))

Okay, here's a structured strategy for effectively learning and mastering Python, SQL, R, Cloud Computing, and Machine Learning Frameworks.  This is designed to be practical and actionable, focusing on building a solid foundation and then specializing.

**Overall Learning Philosophy:**

*   **Project-Based Learning:**  The most effective way to learn is by doing.  Focus on building small, tangible projects that utilize the tools.
*   **Iterative Approach:** Don't try to learn everything at once.  Start with the basics, build a simple project, then expand your knowledge based on the project's needs.  This is much more engaging and effective than passively reading documentation.
*   **Consistent Practice:**  Even 30 minutes of focused practice each day is better than a marathon session once a week.
*   **Community Engagement:**  Join online communities, forums, and attend meetups.  Asking questions, sharing your work, and learning from others is invaluable.
*   **Documentation is Your Fr

In [28]:
chain_long = (chat_template_tools | llm | string_parser | {'tools':RunnablePassthrough()} | 
              chat_template_strategy | llm | string_parser)